# LU Matrix Factorization

## Introduction

LU decomposition (or LU factorization) is a fundamental technique in numerical linear algebra for solving systems of linear equations, computing matrix inverses, and calculating determinants. The method factorizes a matrix $A$ into the product of a lower triangular matrix $L$ and an upper triangular matrix $U$.

## Mathematical Foundation

### Basic Decomposition

Given a square matrix $A \in \mathbb{R}^{n \times n}$, the LU decomposition seeks matrices $L$ and $U$ such that:

$$A = LU$$

where:
- $L$ is a lower triangular matrix with ones on the diagonal (unit lower triangular)
- $U$ is an upper triangular matrix

The elements of $L$ and $U$ are defined as:

$$L = \begin{pmatrix} 1 & 0 & \cdots & 0 \\ l_{21} & 1 & \cdots & 0 \\ \vdots & \vdots & \ddots & \vdots \\ l_{n1} & l_{n2} & \cdots & 1 \end{pmatrix}, \quad U = \begin{pmatrix} u_{11} & u_{12} & \cdots & u_{1n} \\ 0 & u_{22} & \cdots & u_{2n} \\ \vdots & \vdots & \ddots & \vdots \\ 0 & 0 & \cdots & u_{nn} \end{pmatrix}$$

### Doolittle Algorithm

The Doolittle algorithm computes the elements of $L$ and $U$ using the following formulas:

For the elements of $U$ (row $i$, column $j$ where $j \geq i$):
$$u_{ij} = a_{ij} - \sum_{k=1}^{i-1} l_{ik} u_{kj}$$

For the elements of $L$ (row $i$, column $j$ where $i > j$):
$$l_{ij} = \frac{1}{u_{jj}} \left( a_{ij} - \sum_{k=1}^{j-1} l_{ik} u_{kj} \right)$$

### Solving Linear Systems

Once we have $A = LU$, solving $Ax = b$ becomes:

1. **Forward substitution**: Solve $Ly = b$ for $y$
   $$y_i = b_i - \sum_{j=1}^{i-1} l_{ij} y_j$$

2. **Backward substitution**: Solve $Ux = y$ for $x$
   $$x_i = \frac{1}{u_{ii}} \left( y_i - \sum_{j=i+1}^{n} u_{ij} x_j \right)$$

### Partial Pivoting

For numerical stability, we often use partial pivoting, leading to the decomposition:

$$PA = LU$$

where $P$ is a permutation matrix that reorders rows to place the largest pivot element in each column.

### Computational Complexity

The LU decomposition requires $\mathcal{O}\left(\frac{2n^3}{3}\right)$ floating-point operations, making it efficient for solving multiple systems with the same coefficient matrix.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

# Set random seed for reproducibility
np.random.seed(42)

## Implementation of LU Decomposition

We implement the Doolittle algorithm for LU decomposition without pivoting, then compare with SciPy's implementation that includes partial pivoting.

In [ ]:
def lu_decomposition_doolittle(A):
    """
    Perform LU decomposition using the Doolittle algorithm.
    
    Parameters:
    -----------
    A : numpy.ndarray
        Square matrix to decompose
    
    Returns:
    --------
    L : numpy.ndarray
        Lower triangular matrix with unit diagonal
    U : numpy.ndarray
        Upper triangular matrix
    """
    n = A.shape[0]
    L = np.eye(n)
    U = np.zeros((n, n))
    
    for i in range(n):
        # Compute U elements in row i
        for j in range(i, n):
            U[i, j] = A[i, j] - np.sum(L[i, :i] * U[:i, j])
        
        # Compute L elements in column i
        for j in range(i + 1, n):
            if U[i, i] == 0:
                raise ValueError("Zero pivot encountered. Matrix may need pivoting.")
            L[j, i] = (A[j, i] - np.sum(L[j, :i] * U[:i, i])) / U[i, i]
    
    return L, U

In [ ]:
def forward_substitution(L, b):
    """
    Solve Ly = b using forward substitution.
    """
    n = len(b)
    y = np.zeros(n)
    
    for i in range(n):
        y[i] = b[i] - np.sum(L[i, :i] * y[:i])
    
    return y

def backward_substitution(U, y):
    """
    Solve Ux = y using backward substitution.
    """
    n = len(y)
    x = np.zeros(n)
    
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.sum(U[i, i+1:] * x[i+1:])) / U[i, i]
    
    return x

def solve_lu(L, U, b):
    """
    Solve Ax = b using pre-computed LU decomposition.
    """
    y = forward_substitution(L, b)
    x = backward_substitution(U, y)
    return x

## Example: Solving a Linear System

Let's demonstrate the LU decomposition on a $4 \times 4$ matrix and solve a linear system.

In [ ]:
# Create a test matrix
A = np.array([
    [4, 3, 2, 1],
    [3, 4, 3, 2],
    [2, 3, 4, 3],
    [1, 2, 3, 4]
], dtype=float)

# Right-hand side vector
b = np.array([10, 12, 12, 10], dtype=float)

print("Matrix A:")
print(A)
print("\nVector b:")
print(b)

In [ ]:
# Perform LU decomposition
L, U = lu_decomposition_doolittle(A)

print("Lower triangular matrix L:")
print(np.round(L, 4))
print("\nUpper triangular matrix U:")
print(np.round(U, 4))

In [ ]:
# Verify: LU should equal A
LU_product = L @ U
print("L × U:")
print(np.round(LU_product, 4))
print("\nReconstruction error ||A - LU||:", np.linalg.norm(A - LU_product))

In [ ]:
# Solve the system
x = solve_lu(L, U, b)
print("Solution x:")
print(x)

# Verify solution
print("\nVerification Ax:")
print(A @ x)
print("\nResidual ||Ax - b||:", np.linalg.norm(A @ x - b))

## Comparison with SciPy's LU Decomposition

SciPy's `lu` function implements LU decomposition with partial pivoting for improved numerical stability.

In [ ]:
# SciPy's LU decomposition with pivoting
P, L_scipy, U_scipy = linalg.lu(A)

print("Permutation matrix P:")
print(P)
print("\nL (SciPy):")
print(np.round(L_scipy, 4))
print("\nU (SciPy):")
print(np.round(U_scipy, 4))

# Verify: P @ L @ U should equal A
print("\nReconstruction error ||A - PLU||:", np.linalg.norm(A - P @ L_scipy @ U_scipy))

## Visualization: Sparsity Patterns and Performance Analysis

In [ ]:
# Create figure for comprehensive visualization
fig = plt.figure(figsize=(14, 10))

# Matrix sparsity patterns
ax1 = fig.add_subplot(2, 3, 1)
im1 = ax1.imshow(np.abs(A) > 1e-10, cmap='Blues', aspect='equal')
ax1.set_title('Original Matrix A\n(non-zero pattern)', fontsize=11)
ax1.set_xlabel('Column')
ax1.set_ylabel('Row')

ax2 = fig.add_subplot(2, 3, 2)
im2 = ax2.imshow(np.abs(L) > 1e-10, cmap='Greens', aspect='equal')
ax2.set_title('Lower Triangular L\n(non-zero pattern)', fontsize=11)
ax2.set_xlabel('Column')
ax2.set_ylabel('Row')

ax3 = fig.add_subplot(2, 3, 3)
im3 = ax3.imshow(np.abs(U) > 1e-10, cmap='Reds', aspect='equal')
ax3.set_title('Upper Triangular U\n(non-zero pattern)', fontsize=11)
ax3.set_xlabel('Column')
ax3.set_ylabel('Row')

# Heatmaps of actual values
ax4 = fig.add_subplot(2, 3, 4)
im4 = ax4.imshow(A, cmap='coolwarm', aspect='equal')
ax4.set_title('Matrix A (values)', fontsize=11)
ax4.set_xlabel('Column')
ax4.set_ylabel('Row')
plt.colorbar(im4, ax=ax4, shrink=0.8)

ax5 = fig.add_subplot(2, 3, 5)
im5 = ax5.imshow(L, cmap='coolwarm', aspect='equal')
ax5.set_title('Matrix L (values)', fontsize=11)
ax5.set_xlabel('Column')
ax5.set_ylabel('Row')
plt.colorbar(im5, ax=ax5, shrink=0.8)

ax6 = fig.add_subplot(2, 3, 6)
im6 = ax6.imshow(U, cmap='coolwarm', aspect='equal')
ax6.set_title('Matrix U (values)', fontsize=11)
ax6.set_xlabel('Column')
ax6.set_ylabel('Row')
plt.colorbar(im6, ax=ax6, shrink=0.8)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nVisualization saved to plot.png")

## Computational Complexity Analysis

Let's measure the execution time of LU decomposition for matrices of various sizes to observe the $\mathcal{O}(n^3)$ complexity.

In [ ]:
import time

# Test various matrix sizes
sizes = [10, 20, 50, 100, 200, 500]
times_custom = []
times_scipy = []

for n in sizes:
    # Generate random matrix
    A_test = np.random.randn(n, n)
    # Ensure diagonal dominance for stability
    A_test += n * np.eye(n)
    
    # Time custom implementation
    start = time.time()
    for _ in range(3):
        L_test, U_test = lu_decomposition_doolittle(A_test)
    times_custom.append((time.time() - start) / 3)
    
    # Time SciPy implementation
    start = time.time()
    for _ in range(3):
        P_test, L_test, U_test = linalg.lu(A_test)
    times_scipy.append((time.time() - start) / 3)
    
    print(f"n = {n:4d}: Custom = {times_custom[-1]*1000:8.3f} ms, SciPy = {times_scipy[-1]*1000:8.3f} ms")

## Application: Computing Determinants

The determinant of $A$ can be efficiently computed from its LU decomposition:

$$\det(A) = \det(L) \cdot \det(U) = 1 \cdot \prod_{i=1}^{n} u_{ii} = \prod_{i=1}^{n} u_{ii}$$

Since $L$ has unit diagonal, $\det(L) = 1$.

In [ ]:
def determinant_from_lu(U):
    """
    Compute determinant from the U matrix of LU decomposition.
    """
    return np.prod(np.diag(U))

# Compute determinant using LU decomposition
det_lu = determinant_from_lu(U)
det_numpy = np.linalg.det(A)

print(f"Determinant from LU decomposition: {det_lu:.6f}")
print(f"Determinant from NumPy:            {det_numpy:.6f}")
print(f"Relative error: {abs(det_lu - det_numpy) / abs(det_numpy):.2e}")

## Application: Computing Matrix Inverse

The inverse of $A$ can be computed by solving $Ax_i = e_i$ for each standard basis vector $e_i$:

$$A^{-1} = [x_1 | x_2 | \cdots | x_n]$$

This is efficient when we have the LU decomposition, as each solve only requires forward and backward substitution.

In [ ]:
def inverse_from_lu(L, U):
    """
    Compute matrix inverse using LU decomposition.
    """
    n = L.shape[0]
    A_inv = np.zeros((n, n))
    
    for i in range(n):
        # Solve for each column of the inverse
        e_i = np.zeros(n)
        e_i[i] = 1.0
        A_inv[:, i] = solve_lu(L, U, e_i)
    
    return A_inv

# Compute inverse
A_inv = inverse_from_lu(L, U)
print("Inverse of A:")
print(np.round(A_inv, 4))

# Verify: A @ A_inv should be identity
identity_check = A @ A_inv
print("\nA × A⁻¹ (should be identity):")
print(np.round(identity_check, 4))
print(f"\n||A·A⁻¹ - I||: {np.linalg.norm(identity_check - np.eye(4)):.2e}")

## Condition Number and Numerical Stability

The condition number $\kappa(A) = ||A|| \cdot ||A^{-1}||$ indicates how sensitive the solution is to perturbations in the input. A large condition number suggests potential numerical instability.

In [ ]:
# Compute condition number
cond_A = np.linalg.cond(A)
print(f"Condition number of A: {cond_A:.4f}")

# Demonstrate effect of condition number on solution accuracy
# Create an ill-conditioned matrix (Hilbert matrix)
def hilbert_matrix(n):
    return np.array([[1/(i + j + 1) for j in range(n)] for i in range(n)])

print("\nCondition numbers for Hilbert matrices:")
for n in [3, 4, 5, 6, 7, 8]:
    H = hilbert_matrix(n)
    cond_H = np.linalg.cond(H)
    print(f"n = {n}: κ(H) = {cond_H:.2e}")

## Summary

We have explored LU matrix factorization, a cornerstone algorithm in numerical linear algebra:

1. **Theory**: The decomposition $A = LU$ factors a matrix into lower and upper triangular components
2. **Algorithm**: The Doolittle algorithm computes $L$ and $U$ through systematic elimination
3. **Applications**: Efficient solving of linear systems, determinant computation, and matrix inversion
4. **Complexity**: $\mathcal{O}(n^3)$ for decomposition, $\mathcal{O}(n^2)$ for subsequent solves
5. **Stability**: Partial pivoting improves numerical stability for ill-conditioned matrices

LU decomposition is particularly advantageous when solving multiple systems with the same coefficient matrix, as the expensive $\mathcal{O}(n^3)$ factorization is performed once, and each solve requires only $\mathcal{O}(n^2)$ operations.